In [ ]:
import pandas as pd
import seaborn as sns
import warnings
sns.set(style="whitegrid", palette="muted", font_scale=1)
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('~/Downloads/imputed_data/train.csv', index_col='date', infer_datetime_format=True)
df = df_train
df.index = pd.to_datetime(df.index)
df['day'] = df.index.day
df['month'] = df.index.month

In [ ]:
df_test = pd.read_csv('~/Downloads/imputed_data/train.csv', index_col='date', infer_datetime_format=True)
df = df_test
df.index = pd.to_datetime(df.index)
df['day'] = df.index.day
df['month'] = df.index.month

In [ ]:
regions = {
    'West': ['Washington', 'Oregon', 'Idaho', 'Montana', 'Wyoming', 'Arizona', 'New Mexico', 'Oklahoma', 'Texas'],
    'Southwest': ['California', 'Nevada', 'Utah', 'Colorado'],
    'Midwest': ['Illinois', 'Indiana', 'Iowa', 'Kansas', 'Michigan', 'Minnesota',
                'Missouri', 'Nebraska', 'North Dakota', 'Ohio', 'South Dakota', 'Wisconsin'],
    'South': ['Alabama', 'Arkansas', 'Delaware', 'Florida', 'Georgia', 'Kentucky',
              'Louisiana', 'Maryland', 'Mississippi', 'North Carolina', 'South Carolina',
              'Tennessee', 'Virginia', 'West Virginia'],
    'NewEngland': ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont', 'New York', 'New Jersey', 'Pennsylvania'],
    'Utopia': ['Alaska', 'Hawaii']
}

In [ ]:
# Create separate dataframes for each region and set 'date' as the index
West = df[df['state_name'].isin(regions['West'])]
Southwest = df[df['state_name'].isin(regions['Southwest'])]
Midwest = df[df['state_name'].isin(regions['Midwest'])]
NewEngland = df[df['state_name'].isin(regions['NewEngland'])]
South = df[df['state_name'].isin(regions['South'])]
Utopia = df[df['state_name'].isin(regions['Utopia'])]

In [ ]:
# print the number of observations for each region
print('West: ', West.shape[0])
print('Southwest: ', Southwest.shape[0])
print('Midwest: ', Midwest.shape[0])
print('NewEngland: ', NewEngland.shape[0])
print('South: ', South.shape[0])

In [ ]:
# store to csv
# West.to_csv('~/Downloads/imputed_data/West.csv')
# Southwest.to_csv('~/Downloads/imputed_data/Southwest.csv')
# Midwest.to_csv('~/Downloads/imputed_data/Midwest.csv')
# NewEngland.to_csv('~/Downloads/imputed_data/NewEngland.csv')
# South.to_csv('~/Downloads/imputed_data/South.csv')
# Utopia.to_csv('~/Downloads/imputed_data/Utopia.csv')

In [ ]:
def aggregate_monthly(df):
    monthly_df = df.groupby([df.year, df.month]).agg({'covid_19_confirmed_cases':'sum'}).reset_index()
    monthly_df.columns = ['Year', 'Month', 'Monthly_Confirmed_Cases']
    monthly_df['Date'] = pd.to_datetime(monthly_df[['Year', 'Month']].assign(DAY=1))
    monthly_df = monthly_df.set_index('Date')
    return monthly_df

def aggregate_weekly(df):
    weekly_df = df.groupby([df.year, df.month, df.week]).agg({'covid_19_confirmed_cases':'sum'}).reset_index()
    weekly_df.columns = ['Year', 'Month', 'Week', 'Weekly_Confirmed_Cases', '']
    weekly_df['Date'] = pd.to_datetime(weekly_df[['Year', 'Month', 'Week']].assign(DAY=1))
    weekly_df = weekly_df.set_index('Date')
    return weekly_df    